# 정형데이터 vs 비정형데이터  
+ `정형데이터(structured data)` : csv, db, excel에 저장하기 쉬운 구조화된 데이터  
    + __앙상블 학습(ensemble)__: 정형데이터를 다루는 데 가장 성능 좋은 알고리즘
    + 정형 데이터 끝판왕!
+ `비정형데이터(unstructured data)` : 텍스트, 사진, 음악 등  
    + but, 비정형데이터를 데이터 베이스에 저장할 수 없는 건 아님 
    + NoSQL은 텍스트,JSON데이터 저장하는데 용이함
    + __신경망 알고리즘__
        + 비정형 데이터는 규칙성 찾기 어려움
        + 전통 머신러닝으로는 모델 만들기 어려움 

# 랜덤 포레스트 (`sklearn.ensemble.RandomForestClassifier`)  
: 결정트리를 랜덤하게 만들어 트리의 숲을 만든다.  
1. Train Data 생성  
     각각의 트리를 훈련시키기 위한 데이터를 랜덤하게 생성  
    + 훈련세트에서 랜덤 추출한 데이터(복원추출, 훈련세트랑 사이즈 같음)
    + `부스트랩 샘플`(boostrap sample) 
        + ex) 1000개의 샘플 중 1000개를 복원 추출
        +  n(X_train)=n(Boodstrap) 
2. 노드 분할  
전체 Feature 중 $\sqrt{feature~ 총수~~}~~$개의 feature 랜덤하게 선택 후 최선의 분할 찾음  
3. Default 100개의 DecisionTree를 훈련시킴  

<br/>  
+ 장점 : 랜덤하게 선택된 부스트랩 샘플(Boostrap Sample)과 특성(Feature)를 사용하기 때문에  
$⇒$ __train data에 overfitting되는 것을 막아주고__ 안정적인 성능 얻을 수 있음
        

RandomForestClassifier()-  
RandomForestRegressor()

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
wine=pd.read_csv('http://bit.ly/wine_csv_data')
wine.head()

,alcohol,sugar,pH,class
0,9.4,1.9,3.51,0.0
1,9.8,2.6,3.20,0.0
2,9.8,2.3,3.26,0.0
3,9.8,1.9,3.16,0.0
4,9.4,1.9,3.51,0.0


In [3]:
wine.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6497 entries, 0 to 6496
Data columns (total 4 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   alcohol  6497 non-null   float64
 1   sugar    6497 non-null   float64
 2   pH       6497 non-null   float64
 3   class    6497 non-null   float64
dtypes: float64(4)
memory usage: 203.2 KB


In [4]:
X=wine[['alcohol','sugar','pH']]
y=wine['class']

X.shape, y.shape

((6497, 3), (6497,))

In [5]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

X_train.shape, y_train.shape, X_test.shape, y_test.shape

((5197, 3), (5197,), (1300, 3), (1300,))

## RandomForestClassifier() 사용해보기

In [6]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_jobs=-1,random_state=42)
rf.fit(X_train,y_train)

RandomForestClassifier(n_jobs=-1, random_state=42)

In [7]:
print(f'훈련 점수 : {rf.score(X_train,y_train)}')
print(f'테스트 점수 : {rf.score(X_test,y_test)}')

훈련 점수 : 0.996921300750433
테스트 점수 : 0.8892307692307693


In [8]:
rf.base_estimator_ # The child estimator template used

DecisionTreeClassifier()

In [9]:
rf.estimators_ # The collection of fitted sub-estimators.

[DecisionTreeClassifier(max_features='auto', random_state=1608637542),
 DecisionTreeClassifier(max_features='auto', random_state=1273642419),
 DecisionTreeClassifier(max_features='auto', random_state=1935803228),
 DecisionTreeClassifier(max_features='auto', random_state=787846414),
 DecisionTreeClassifier(max_features='auto', random_state=996406378),
 DecisionTreeClassifier(max_features='auto', random_state=1201263687),
 DecisionTreeClassifier(max_features='auto', random_state=423734972),
 DecisionTreeClassifier(max_features='auto', random_state=415968276),
 DecisionTreeClassifier(max_features='auto', random_state=670094950),
 DecisionTreeClassifier(max_features='auto', random_state=1914837113),
 DecisionTreeClassifier(max_features='auto', random_state=669991378),
 DecisionTreeClassifier(max_features='auto', random_state=429389014),
 DecisionTreeClassifier(max_features='auto', random_state=249467210),
 DecisionTreeClassifier(max_features='auto', random_state=1972458954),
 DecisionTreeC

In [10]:
len(rf.estimators_) # 100개

100

In [11]:
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

## cross_validate() 교차검증

In [12]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
    # RFC는 기본적으로 100개의 결정트리를 사용

rf=RandomForestClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(rf,X_train,y_train,
                    return_train_score=True,n_jobs=-1)
                    # return_train_score=True 지정해주면 훈련 세트 점수도 반환
                    # cross_validate() default cv=5

scores

{'fit_time': array([0.45433164, 0.45443892, 0.44732785, 0.44969296, 0.5674026 ]),
 'score_time': array([0.10310984, 0.10301995, 0.10324597, 0.10329342, 0.11157918]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])}

In [13]:
print(f"훈련 점수 : {scores['train_score']}")
print(f"검증 점수 : {scores['test_score']}")

훈련 점수 : [0.9971133  0.99663219 0.9978355  0.9973545  0.9978355 ]
검증 점수 : [0.88461538 0.88942308 0.90279115 0.88931665 0.88642926]


`RandomForestClassifier()` 를 통해 100개의 decision tree 만듬  
> -> 어찌어찌 해서 1개의 분류모델 결정  

`cross_validate(cv=5)` 이므로 5개의 폴드로 RCF로 결정된 하나의 모델 교차검증 수행  
> -> 각 검증 마다 훈련점수/검증점수 계산됨

In [14]:
print(f"훈련 점수 평균 : {np.mean(scores['train_score'])}")
print(f"검증 점수 평균 : {np.mean(scores['test_score'])}")

훈련 점수 평균 : 0.9973541965122431
검증 점수 평균 : 0.8905151032797809


Overfitting!

In [15]:
rf.fit(X_train,y_train)
rf.feature_importances_

array([0.23167441, 0.50039841, 0.26792718])

`rf.fit(X_train,y_train)` 해주는 이유 :  
cross_validate는 단지 모델의 점수 계산만 해준다 -> 모델 피팅 해줘야함!

In [16]:
# DecisionTreeClassifier()과 피처 중요도 비교하기
from sklearn.tree import DecisionTreeClassifier
dt=DecisionTreeClassifier(random_state=42,max_depth=3)
dt.fit(X_train,y_train)

print(dt.feature_names_in_)
dt.feature_importances_

['alcohol' 'sugar' 'pH']


array([0.12345626, 0.86862934, 0.0079144 ])

`sugar` 피처의 중요도 줄어듬  
$⇒$ RandomForestClassifier 모델은 특정 feature에 과도하게 집중하지 않고 더 많은 특성이 훈련에 기여함  
$⇒$ 일반화 성능 상승!

### OOB 샘플(Out of Bag)  
 RandomForestClassifier는 자체적으로 모델을 평가하는 기능이 있다!  
 Boostrap sample에 포함되지 않은 샘플을 __OOB Sample__이라고 함  
 + OOB Sample로 모델 평가 
 + `RandomForestClassifier(oob_score=True)`  
    + 100개의 DecisionTreeClassifier의 OOB 점수를 평균하여 출력
    + 즉, __자체적인 Validation Score__를 매길 수 있음
    + 교차검증을 대신함

In [17]:
rf=RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(X_train,y_train)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리 (`sklearn.ensemble.ExtraTreesClassifier`)  
: RandomForestClassifier와 유사  
+ 트리들을 훈련할때 Boostrap Sample을 사용하지 않고 전체 train data를 사용 
+ 노드 분할 시 랜덤하게 분할
    + `DecisionTreeClassifier(splitter='random')` 을 100개 만드는 거랑 같음
    + default splitter='best'
    + 최적의 노드 분할이 더 좋은 것 아닌가?
        + 하나의 모델의 경우 최적의 노드 분할이 더 좋은 성능으로 이어지겠지만,   
        __많은 트리를 앙상블하기 때문에 Overfitting을 막을 수 있다__

In [18]:
dt_random=DecisionTreeClassifier(splitter='random',random_state=42,max_depth=3)
dt_random.fit(X_train,y_train)

print('----------- splitter="random"----------')
print(f'훈련 세트 점수 : {dt_random.score(X_train,y_train)}')
print(f'테스트 세트 점수: {dt_random.score(X_test,y_test)}')

print('\n----------- splitter="best"-----------')
dt_best=DecisionTreeClassifier(splitter='best',random_state=42,max_depth=3)
dt_best.fit(X_train,y_train)
print(f'훈련 세트 점수 : {dt_best.score(X_train,y_train)}')
print(f'테스트 세트 점수: {dt_best.score(X_test,y_test)}')

----------- splitter="random"----------
훈련 세트 점수 : 0.7644795074081201
테스트 세트 점수: 0.7407692307692307

----------- splitter="best"-----------
훈련 세트 점수 : 0.8454877814123533
테스트 세트 점수: 0.8415384615384616


In [19]:
from sklearn.ensemble import ExtraTreesClassifier
et=ExtraTreesClassifier(n_jobs=-1,random_state=42)
scores=cross_validate(et,X_train,y_train,
                      return_train_score=True, n_jobs=-1)

scores

{'fit_time': array([0.65706396, 0.48658228, 0.50507259, 0.40448594, 0.35012698]),
 'score_time': array([0.11646366, 0.14288759, 0.22615409, 0.2232635 , 0.10398006]),
 'test_score': array([0.88365385, 0.87884615, 0.90375361, 0.88835419, 0.88931665]),
 'train_score': array([0.9971133 , 0.99663219, 0.998076  , 0.997595  , 0.9978355 ])}

In [20]:
print(f'Train 점수 : {np.mean(scores["train_score"])}')
print(f'Validation 점수 : {np.mean(scores["test_score"])}')

Train 점수 : 0.9974503966084433
Validation 점수 : 0.8887848893166506


In [21]:
et.fit(X_train,y_train)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]




```
## RandomForestClassifier 성능
 {'fit_time': array([0.57525206, 0.5712173 , 0.58535051, 0.56668949, 0.43762898]),
 'score_time': array([0.10490441, 0.10518217, 0.10390687, 0.10453987, 0.10560322]),
 'test_score': array([0.88461538, 0.88942308, 0.90279115, 0.88931665, 0.88642926]),
 'train_score': array([0.9971133 , 0.99663219, 0.9978355 , 0.9973545 , 0.9978355 ])} 
 # 요약
  훈련 점수 평균 : 0.9973541965122431
  검증 점수 평균 : 0.8905151032797809

## ExtraTreeClassifier 성능
 {'fit_time': array([0.58513284, 0.48433852, 0.46698499, 0.46554995, 0.44021702]),
 'score_time': array([0.10926461, 0.10540247, 0.11381841, 0.10430717, 0.10439658]),
 'test_score': array([0.88365385, 0.87884615, 0.90375361, 0.88835419, 0.88931665]),
 'train_score': array([0.9971133 , 0.99663219, 0.998076  , 0.997595  , 0.9978355 ])}
 # 요약
 Train 점수 : 0.9974503966084433
 Validation 점수 : 0.8887848893166506
```



+ ETC 모델은 노드 분할을 랜덤으로 하기때문에 RFC 모델보다 학습 시간이 더 짧음

# 그래디언트 부스팅 (`sklearn.ensemble.GradientBoostingClassifier`)  


In [22]:
from sklearn.ensemble import GradientBoostingClassifier
gb=GradientBoostingClassifier(random_state=42)
scores=cross_validate(gb,X_train,y_train,
                      return_train_score=True,n_jobs=-1)

scores

{'fit_time': array([0.62190604, 0.72479701, 0.71795845, 0.67013097, 0.48699617]),
 'score_time': array([0.00434303, 0.00404263, 0.00459886, 0.00476527, 0.00465155]),
 'test_score': array([0.86634615, 0.87019231, 0.89412897, 0.86044273, 0.86910491]),
 'train_score': array([0.89006495, 0.88958383, 0.88239538, 0.89249639, 0.88600289])}

In [23]:
print(f'Train 점수 : {np.mean(scores["train_score"])}')
print(f'Validation 점수 : {np.mean(scores["test_score"])}')

Train 점수 : 0.8881086892152563
Validation 점수 : 0.8720430147331015


__과대적합 해결!!__

In [24]:
gb=GradientBoostingClassifier(n_estimators=500, # 100개 대신 500개의 결정트리 생성
                              learning_rate=0.2, # 학습률 증가 (default=0.1)
                              random_state=42)
scores=cross_validate(gb,X_train,y_train,
                      return_train_score=True,n_jobs=-1)
scores

{'fit_time': array([3.04231215, 3.42220497, 2.31366611, 2.21687222, 1.46597362]),
 'score_time': array([0.01544952, 0.0389533 , 0.01139998, 0.01136065, 0.01136494]),
 'test_score': array([0.875     , 0.87211538, 0.89701636, 0.8719923 , 0.87391723]),
 'train_score': array([0.9494828 , 0.94443108, 0.94468494, 0.94324194, 0.95045695])}

In [25]:
print(f'Train 점수 : {np.mean(scores["train_score"])}')
print(f'Validation 점수 : {np.mean(scores["test_score"])}')

Train 점수 : 0.9464595437171814
Validation 점수 : 0.8780082549788999


# 히스토그램 기반 그레디언트 부스팅  
:Histogram-based Gradient Boosting  
+ `n_estimators` 대신 `max_iter` 로 부스팅 반복 횟수 지정  

In [26]:
from sklearn.ensemble import HistGradientBoostingClassifier
hgb=HistGradientBoostingClassifier(random_state=42)
scores=cross_validate(hgb,X_train,y_train,
                      return_train_score=True)

In [28]:
scores

{'fit_time': array([0.34467244, 0.27167082, 0.26795673, 0.28400826, 0.28369331]),
 'score_time': array([0.00996041, 0.00938988, 0.00970244, 0.01087618, 0.00939369]),
 'test_score': array([0.87115385, 0.88365385, 0.90279115, 0.86621752, 0.87680462]),
 'train_score': array([0.93192206, 0.93216262, 0.92857143, 0.93265993, 0.93554594])}

In [31]:
print(f'Train 점수 : {np.mean(scores["train_score"])}')
print(f'Validation 점수 : {np.mean(scores["test_score"])}')

Train 점수 : 0.9321723946453317
Validation 점수 : 0.8801241948619236


# Permutation Importance

In [32]:
from sklearn.inspection import permutation_importance
hgb.fit(X_train,y_train)
result=permutation_importance(hgb,X_train,y_train,
                             n_repeats=10,random_state=42,n_jobs=-1)

In [33]:
print(result)

{'importances_mean': array([0.08876275, 0.23438522, 0.08027708]), 'importances_std': array([0.00382333, 0.00401363, 0.00477012]), 'importances': array([[0.08793535, 0.08350972, 0.08908986, 0.08312488, 0.09274581,
        0.08755051, 0.08601116, 0.09601693, 0.09082163, 0.09082163],
       [0.22782374, 0.23590533, 0.23936887, 0.23436598, 0.23725226,
        0.23436598, 0.23359631, 0.23398114, 0.23994612, 0.22724649],
       [0.08581874, 0.08601116, 0.08062344, 0.07504329, 0.08427939,
        0.07792957, 0.07234943, 0.07465846, 0.08139311, 0.08466423]])}


In [35]:
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]
